In [26]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

import compute_charge_dist as fz

from scipy.optimize import curve_fit

import time

%matplotlib inline


grain_type = "silicate"
#grain_type = "carbonaceous"
pcent      = 1

include_500_1000 = True

########################################################################################

grain_size = 3

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_3AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 5

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_5AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 10

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_10AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 50

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_50AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 100

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_100AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

if include_500_1000 == True:
    print("Loading the charge distribution for 500 and 1000 Angstrom grains")
    grain_size = 500
    pcent      = 0.1

    save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
    filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

    pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
    cdist_500AA = pickle.load(pkl_file)
    pkl_file.close

    ########################################################################################

    grain_size = 1000

    save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
    filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

    pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
    cdist_1000AA = pickle.load(pkl_file)
    pkl_file.close


Loading the charge distribution for 500 and 1000 Angstrom grains


In [27]:
too_high = np.where(cdist_3AA["nH"] > 2.0e4)
if include_500_1000 == True:
    too_high_500 = np.where(cdist_500AA["nH"] > 2.0e4)
    too_high_1000 = np.where(cdist_1000AA["nH"] > 2.0e4)
    
ii = 0
for key in cdist_3AA.keys():
    #arrs.append(0)
    if key!="Qabs_mean" and key!="grain_size" and key!="grain_type" and key!="MPI" and key!="fdist" and key!="info" and key!="zminmax" and key!="ZZ":
        #print(key)
        cdist_3AA[key]    = np.delete(cdist_3AA[key],    too_high)
        cdist_5AA[key]    = np.delete(cdist_5AA[key],    too_high)
        cdist_10AA[key]   = np.delete(cdist_10AA[key],   too_high)
        cdist_50AA[key]   = np.delete(cdist_50AA[key],   too_high)
        cdist_100AA[key]  = np.delete(cdist_100AA[key],  too_high)
        if include_500_1000 == True:
            cdist_500AA[key]  = np.delete(cdist_500AA[key],  too_high_500)
            cdist_1000AA[key] = np.delete(cdist_1000AA[key], too_high_1000)
    #ii +=1

In [28]:
GTn_3AA    = (cdist_3AA["G"]  +cdist_3AA["G_CR"])  * np.sqrt(cdist_3AA["temp"])   / cdist_3AA["new_ne"] 
GTn_5AA    = (cdist_5AA["G"]  +cdist_5AA["G_CR"])  * np.sqrt(cdist_5AA["temp"])   / cdist_5AA["new_ne"] 
GTn_10AA   = (cdist_10AA["G"] +cdist_10AA["G_CR"]) * np.sqrt(cdist_10AA["temp"])  / cdist_10AA["new_ne"] 
GTn_50AA   = (cdist_50AA["G"] +cdist_50AA["G_CR"]) * np.sqrt(cdist_50AA["temp"])  / cdist_50AA["new_ne"] 
GTn_100AA  = (cdist_100AA["G"]+cdist_100AA["G_CR"])* np.sqrt(cdist_100AA["temp"]) / cdist_100AA["new_ne"] 

if include_500_1000 == True:
    GTn_500AA  = (cdist_500AA["G"] +cdist_500AA["G_CR"])  * np.sqrt(cdist_500AA["temp"])  / cdist_500AA["new_ne"] 
    GTn_1000AA = (cdist_1000AA["G"]+cdist_1000AA["G_CR"]) * np.sqrt(cdist_1000AA["temp"]) / cdist_1000AA["new_ne"] 

In [29]:
# New function like a press schechter like, combining a power law and an exponential growth.

# New function including powers between -3, 3
# def function(x,alpha1, beta1, gamma1, alpha2, beta2, gamma2):
def function(x, alpha, aa, bb, hh):

    Zcent = aa * (x[:])**alpha * (1.0 - np.exp(-x[:]/hh)) + bb
    
    return Zcent

In [54]:
Zparam_3AA   = function(GTn_3AA, 0.3263, 0.0149, -0.1212, 57. )
Zparam_5AA   = function(GTn_5AA, 0.3141, 0.0372, -0.3043, 86.)
Zparam_10AA  = function(GTn_10AA, 0.3535, 0.0494, -0.4865, 73.)
Zparam_50AA  = function(GTn_50AA, 0.5220, 0.0713, -0.6221, 12.8305)
#Zparam_50AA  = function(GTn_50AA, 0.4820, 0.0913, -0.6221, 10.8305)
#Zparam_100AA = function(GTn_100AA, 0.3525, 0.6591, -0.1649, 384.)
Zparam_100AA = function(GTn_100AA, 2.88641830e-01,   1.24494490e+00,  -1.34919127e-01,   6.56732059e+02)

Zparam_500AA  = function(GTn_500AA, 0.3643, 2.6283, 0.5217, 345.)
Zparam_1000AA = function(GTn_1000AA, 0.3927, 3.6493, 0.8389, 372.)

In [42]:
Zparam_3AA   = function(GTn_3AA, 0.3265, 0.0149, -0.1212, 56.9630)
Zparam_5AA   = function(GTn_5AA, 0.3141, 0.0372, -0.3043, 85.6275)
Zparam_10AA  = function(GTn_10AA, 0.3535, 0.0494, -0.4865, 72.7797 )
Zparam_50AA  = function(GTn_50AA, 0.5220, 0.0713, -0.6221, 12.8305)
Zparam_100AA = function(GTn_100AA, 0.3682, 0.5717, -0.4038, 290.1160)
Zparam_500AA  = function(GTn_500AA, 0.3672, 2.5551, -0.1326, 301.3965)
Zparam_1000AA = function(GTn_1000AA, 0.3965, 3.5180, -0.0333, 331.4851)

In [58]:
# GTn small = 1
Zparam_3AA   = function(GTn_3AA, 0.4100,0.0068, -0.0944, 108.6085)
Zparam_5AA   = function(GTn_5AA, 0.3698, 0.0215, -0.2475, 161.6249 )
Zparam_10AA  = function(GTn_10AA,0.3751, 0.0391, -0.4352, 111.1735  )
Zparam_50AA  = function(GTn_50AA, 0.5097, 0.0801, -0.6877, 5.9270 )
Zparam_100AA = function(GTn_100AA, 0.3844, 0.5007, -0.5433, 234.8363 )
Zparam_500AA  = function(GTn_500AA, 0.3699, 2.4956, -0.4780, 278.6272 )
Zparam_1000AA = function(GTn_1000AA, 0.3989, 3.4420, -0.4416, 312.4704 )

In [62]:
print("                SILICATE ")
print("===============================================")
print("Error")
print("size \t 25% \t 50% \t 75%  ")
print("%.1f &\t %.1f &\t %f &\t %f \\"%(3.5, percentile_3AA[0], percentile_3AA[1], percentile_3AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(5,percentile_5AA[0], percentile_5AA[1], percentile_5AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(10,percentile_10AA[0], percentile_10AA[1], percentile_10AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(50,percentile_50AA[0], percentile_50AA[1], percentile_50AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(100,percentile_100AA[0], percentile_100AA[1], percentile_100AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(500,percentile_500AA[0], percentile_500AA[1], percentile_500AA[2]))
print("%.0f &\t %.1f &\t %f &\t %f \\"%(1000, percentile_1000AA[0], percentile_1000AA[1], percentile_1000AA[2]))

                SILICATE 
Error
size 	 25% 	 50% 	 75%  
3.5 &	 6.5 &	 15.768693 &	 27.579963 \
5.0 &	 7.2 &	 11.222284 &	 19.185754 \
10.0 &	 1.8 &	 5.098720 &	 10.453573 \
50.0 &	 5.8 &	 11.992466 &	 17.007911 \
100.0 &	 4.3 &	 10.217937 &	 48.153776 \
500.0 &	 2.4 &	 7.018643 &	 28.763797 \
1000 &	 2.4 &	 7.145655 &	 27.497203 \


In [63]:
# GTn small = 10
Zparam_3AA   = function(GTn_3AA, 0.3265, 0.0149, -0.1212, 56.9630 )
Zparam_5AA   = function(GTn_5AA, 0.3141, 0.0372, -0.3043, 85.6275)
Zparam_10AA  = function(GTn_10AA,0.3535, 0.0494, -0.4865, 72.7797 )
Zparam_50AA  = function(GTn_50AA, 0.5220, 0.0713, -0.6221, 12.8305 )
Zparam_100AA = function(GTn_100AA, 0.3682, 0.5717, -0.4038, 290.1160 )
Zparam_500AA  = function(GTn_500AA, 0.3672, 2.5551, -0.1326, 301.3965 )
Zparam_1000AA = function(GTn_1000AA, 0.3965, 3.5180, -0.0333, 331.4851 )

In [66]:
print("                SILICATE ")
print("===============================================")
print("Error")
print("size \t 25% \t 50% \t 75%  ")
print("%.1f &\t %.1f &\t %f &\t %f \\"%(3.5, percentile_3AA[0], percentile_3AA[1], percentile_3AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(5,percentile_5AA[0], percentile_5AA[1], percentile_5AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(10,percentile_10AA[0], percentile_10AA[1], percentile_10AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(50,percentile_50AA[0], percentile_50AA[1], percentile_50AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(100,percentile_100AA[0], percentile_100AA[1], percentile_100AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(500,percentile_500AA[0], percentile_500AA[1], percentile_500AA[2]))
print("%.0f &\t %.1f &\t %f &\t %f \\"%(1000, percentile_1000AA[0], percentile_1000AA[1], percentile_1000AA[2]))

                SILICATE 
Error
size 	 25% 	 50% 	 75%  
3.5 &	 5.4 &	 13.088541 &	 20.128156 \
5.0 &	 2.4 &	 4.411067 &	 9.144792 \
10.0 &	 1.3 &	 3.878859 &	 7.739508 \
50.0 &	 6.0 &	 13.174081 &	 18.283164 \
100.0 &	 3.9 &	 9.147263 &	 33.151210 \
500.0 &	 2.1 &	 6.383438 &	 20.390748 \
1000 &	 2.2 &	 6.563527 &	 18.525792 \


In [67]:
# GTn small = =50
Zparam_3AA   = function(GTn_3AA, 0.3263, 0.0149, -0.1212, 57.1367)
Zparam_5AA   = function(GTn_5AA, 0.3141, 0.0372 , -0.3043, 85.6420  )
Zparam_10AA  = function(GTn_10AA,0.3535, 0.0494, -0.4865, 72.7792 )
Zparam_50AA  = function(GTn_50AA, 0.5115, 0.0717, -0.4106, 107.5547 )
Zparam_100AA = function(GTn_100AA, 0.3525, 0.6591, -0.1649, 383.8483 )
Zparam_500AA  = function(GTn_500AA, 0.3643, 2.6284, 0.5217, 344.8246 )
Zparam_1000AA = function(GTn_1000AA,  0.3927, 3.6494, 0.8389, 372.4563)

In [70]:
print("                SILICATE ")
print("===============================================")
print("Error")
print("size \t 25% \t 50% \t 75%  ")
print("%.1f &\t %.1f &\t %f &\t %f \\"%(3.5, percentile_3AA[0], percentile_3AA[1], percentile_3AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(5,percentile_5AA[0], percentile_5AA[1], percentile_5AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(10,percentile_10AA[0], percentile_10AA[1], percentile_10AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(50,percentile_50AA[0], percentile_50AA[1], percentile_50AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(100,percentile_100AA[0], percentile_100AA[1], percentile_100AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(500,percentile_500AA[0], percentile_500AA[1], percentile_500AA[2]))
print("%.0f &\t %.1f &\t %f &\t %f \\"%(1000, percentile_1000AA[0], percentile_1000AA[1], percentile_1000AA[2]))

                SILICATE 
Error
size 	 25% 	 50% 	 75%  
3.5 &	 5.4 &	 13.162389 &	 20.048709 \
5.0 &	 2.4 &	 4.410871 &	 9.144820 \
10.0 &	 1.3 &	 3.878859 &	 7.739358 \
50.0 &	 7.8 &	 14.263777 &	 21.677842 \
100.0 &	 2.6 &	 7.557571 &	 32.809872 \
500.0 &	 1.5 &	 3.667941 &	 9.425229 \
1000 &	 1.5 &	 4.063206 &	 9.738040 \


In [71]:
# no GTNsmall
Zparam_3AA   = function(GTn_3AA, 0.3264, 0.0149, -0.1212, 57.0676 )
Zparam_5AA   = function(GTn_5AA, 0.3141, 0.0372, -0.3043, 85.6226  )
Zparam_10AA  = function(GTn_10AA,0.3535, 0.0494, -0.4865, 72.7794 )
Zparam_50AA  = function(GTn_50AA, 0.4815, 0.1025, -0.8035, 0.0029)
Zparam_100AA = function(GTn_100AA, 0.3527, 0.6577, -0.1703, 381.7488 )
Zparam_500AA  = function(GTn_500AA, 0.3652, 2.6048, 0.2545, 327.0657  )
Zparam_1000AA = function(GTn_1000AA, 0.3938, 3.6093, 0.5418, 358.4490 )

In [74]:
print("                SILICATE ")
print("===============================================")
print("Error")
print("size \t 25% \t 50% \t 75%  ")
print("%.1f &\t %.1f &\t %f &\t %f \\"%(3.5, percentile_3AA[0], percentile_3AA[1], percentile_3AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(5,percentile_5AA[0], percentile_5AA[1], percentile_5AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(10,percentile_10AA[0], percentile_10AA[1], percentile_10AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(50,percentile_50AA[0], percentile_50AA[1], percentile_50AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(100,percentile_100AA[0], percentile_100AA[1], percentile_100AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(500,percentile_500AA[0], percentile_500AA[1], percentile_500AA[2]))
print("%.0f &\t %.1f &\t %f &\t %f \\"%(1000, percentile_1000AA[0], percentile_1000AA[1], percentile_1000AA[2]))

                SILICATE 
Error
size 	 25% 	 50% 	 75%  
3.5 &	 5.4 &	 13.127212 &	 20.075687 \
5.0 &	 2.4 &	 4.410919 &	 9.145244 \
10.0 &	 1.3 &	 3.878859 &	 7.739418 \
50.0 &	 5.8 &	 11.678939 &	 16.214881 \
100.0 &	 2.6 &	 7.587660 &	 32.811236 \
500.0 &	 1.6 &	 4.470797 &	 8.880890 \
1000 &	 1.7 &	 4.536687 &	 9.244090 \


##### Final combination of parameters.

In [103]:
# no GTNsmall
Zparam_3AA   = function(GTn_3AA, 0.3265, 0.0149, -0.1212, 56.9630)
Zparam_5AA   = function(GTn_5AA, 0.3141, 0.0372 , -0.3043, 85.6420)
Zparam_10AA  = function(GTn_10AA,0.3535, 0.0494, -0.4865, 72.7797 ) 
Zparam_50AA  = function(GTn_50AA, 0.5115, 0.0717, -0.4106, 107.5547)
Zparam_100AA = function(GTn_100AA, 0.3525, 0.6591, -0.1649, 383.8483)
Zparam_500AA  = function(GTn_500AA, 0.3643, 2.6284, 0.5217, 344.8246)
Zparam_1000AA = function(GTn_1000AA,  0.3927, 3.6494, 0.8389, 372.4563)

In [104]:
rel_error_3AA    = abs(Zparam_3AA    - cdist_3AA["zmean"])/abs(cdist_3AA["zmean"])
rel_error_5AA    = abs(Zparam_5AA    - cdist_5AA["zmean"])/abs(cdist_5AA["zmean"])
rel_error_10AA   = abs(Zparam_10AA   - cdist_10AA["zmean"])/abs(cdist_10AA["zmean"])
rel_error_50AA   = abs(Zparam_50AA   - cdist_50AA["zmean"])/abs(cdist_50AA["zmean"])
rel_error_100AA  = abs(Zparam_100AA  - cdist_100AA["zmean"])/abs(cdist_100AA["zmean"])
rel_error_500AA  = abs(Zparam_500AA  - cdist_500AA["zmean"])/abs(cdist_500AA["zmean"])
rel_error_1000AA = abs(Zparam_1000AA - cdist_1000AA["zmean"])/abs(cdist_1000AA["zmean"])

In [105]:
percentile_3AA = np.percentile(rel_error_3AA, [25, 50, 75])*100
percentile_5AA = np.percentile(rel_error_5AA, [25, 50, 75])*100
percentile_10AA = np.percentile(rel_error_10AA, [25, 50, 75])*100
percentile_50AA = np.percentile(rel_error_50AA, [25, 50, 75])*100
percentile_100AA = np.percentile(rel_error_100AA, [25, 50, 75])*100
percentile_500AA = np.percentile(rel_error_500AA, [25, 50, 75])*100
percentile_1000AA = np.percentile(rel_error_1000AA, [25, 50, 75])*100

In [106]:
print("                SILICATE ")
print("===============================================")
print("Error")
print("size \t 25% \t 50% \t 75%  ")
print("%.1f &\t %.1f &\t %f &\t %f \\"%(3.5, percentile_3AA[0], percentile_3AA[1], percentile_3AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(5,percentile_5AA[0], percentile_5AA[1], percentile_5AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(10,percentile_10AA[0], percentile_10AA[1], percentile_10AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(50,percentile_50AA[0], percentile_50AA[1], percentile_50AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(100,percentile_100AA[0], percentile_100AA[1], percentile_100AA[2]))
print("%.1f &\t %.1f &\t %f &\t %f \\"%(500,percentile_500AA[0], percentile_500AA[1], percentile_500AA[2]))
print("%.0f &\t %.1f &\t %f &\t %f \\"%(1000, percentile_1000AA[0], percentile_1000AA[1], percentile_1000AA[2]))

                SILICATE 
Error
size 	 25% 	 50% 	 75%  
3.5 &	 5.4 &	 13.088541 &	 20.128156 \
5.0 &	 2.4 &	 4.410871 &	 9.144820 \
10.0 &	 1.3 &	 3.878859 &	 7.739508 \
50.0 &	 7.8 &	 14.263777 &	 21.677842 \
100.0 &	 2.6 &	 7.557571 &	 32.809872 \
500.0 &	 1.5 &	 3.667941 &	 9.425229 \
1000 &	 1.5 &	 4.063206 &	 9.738040 \


---


In [109]:
#grain_type = "silicate"
grain_type = "carbonaceous"
pcent      = 1

include_500_1000 = True

########################################################################################

grain_size = 3

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_3AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 5

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_5AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 10

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_10AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 50

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_50AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

grain_size = 100

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist_100AA = pickle.load(pkl_file)
pkl_file.close

########################################################################################

if include_500_1000 == True:
    print("Loading the charge distribution for 500 and 1000 Angstrom grains")
    grain_size = 500
    pcent      = 0.1

    save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
    filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

    pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
    cdist_500AA = pickle.load(pkl_file)
    pkl_file.close

    ########################################################################################

    grain_size = 1000

    save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
    filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

    pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
    cdist_1000AA = pickle.load(pkl_file)
    pkl_file.close


too_high = np.where(cdist_3AA["nH"] > 2.0e4)
if include_500_1000 == True:
    too_high_500 = np.where(cdist_500AA["nH"] > 2.0e4)
    too_high_1000 = np.where(cdist_1000AA["nH"] > 2.0e4)
    
ii = 0
for key in cdist_3AA.keys():
    #arrs.append(0)
    if key!="Qabs_mean" and key!="grain_size" and key!="grain_type" and key!="MPI" and key!="fdist" and key!="info" and key!="zminmax" and key!="ZZ":
        #print(key)
        cdist_3AA[key]    = np.delete(cdist_3AA[key],    too_high)
        cdist_5AA[key]    = np.delete(cdist_5AA[key],    too_high)
        cdist_10AA[key]   = np.delete(cdist_10AA[key],   too_high)
        cdist_50AA[key]   = np.delete(cdist_50AA[key],   too_high)
        cdist_100AA[key]  = np.delete(cdist_100AA[key],  too_high)
        if include_500_1000 == True:
            cdist_500AA[key]  = np.delete(cdist_500AA[key],  too_high_500)
            cdist_1000AA[key] = np.delete(cdist_1000AA[key], too_high_1000)
    #ii +=1
    

        
GTn_3AA    = (cdist_3AA["G"]   + cdist_3AA["G_CR"]) * np.sqrt(cdist_3AA["temp"])   / cdist_3AA["new_ne"] 
GTn_5AA    = (cdist_5AA["G"]   + cdist_5AA["G_CR"]) * np.sqrt(cdist_5AA["temp"])   / cdist_5AA["new_ne"] 
GTn_10AA   = (cdist_10AA["G"]  + cdist_10AA["G_CR"]) * np.sqrt(cdist_10AA["temp"])  / cdist_10AA["new_ne"] 
GTn_50AA   = (cdist_50AA["G"]  + cdist_50AA["G_CR"]) * np.sqrt(cdist_50AA["temp"])  / cdist_50AA["new_ne"] 
GTn_100AA  = (cdist_100AA["G"] + cdist_100AA["G_CR"]) * np.sqrt(cdist_100AA["temp"]) / cdist_100AA["new_ne"] 

if include_500_1000 == True:
    GTn_500AA  = (cdist_500AA["G"] + cdist_500AA["G_CR"]) * np.sqrt(cdist_500AA["temp"])  / cdist_500AA["new_ne"] 
    GTn_1000AA = (cdist_1000AA["G"]+ cdist_1000AA["G_CR"]) * np.sqrt(cdist_1000AA["temp"]) / cdist_1000AA["new_ne"]


Loading the charge distribution for 500 and 1000 Angstrom grains


In [133]:
Zparam_3AA   = function(GTn_3AA, 0.4699, 0.0085, -0.1162, 48. )
Zparam_5AA   = function(GTn_5AA, 0.4386, 0.0195, -0.3084, 95. )
Zparam_10AA  = function(GTn_10AA,0.4994, 0.0199, -0.4959, 78. )
Zparam_50AA  = function(GTn_50AA, 0.6009, 0.0523, -0.4092, 218.)
Zparam_100AA = function(GTn_100AA, 0.1021, 14.0536, -0.2519, 2349. )
Zparam_500AA  = function(GTn_500AA, 0.1897, 25.4864, 0.1528, 1974.)
Zparam_1000AA = function(GTn_1000AA, 0.4601, 3.2625, 0.4184, 764.)

In [134]:
rel_error_3AA    = abs(Zparam_3AA    - cdist_3AA["zmean"])/abs(cdist_3AA["zmean"])
rel_error_5AA    = abs(Zparam_5AA    - cdist_5AA["zmean"])/abs(cdist_5AA["zmean"])
rel_error_10AA   = abs(Zparam_10AA   - cdist_10AA["zmean"])/abs(cdist_10AA["zmean"])
rel_error_50AA   = abs(Zparam_50AA   - cdist_50AA["zmean"])/abs(cdist_50AA["zmean"])
rel_error_100AA  = abs(Zparam_100AA  - cdist_100AA["zmean"])/abs(cdist_100AA["zmean"])
rel_error_500AA  = abs(Zparam_500AA  - cdist_500AA["zmean"])/abs(cdist_500AA["zmean"])
rel_error_1000AA = abs(Zparam_1000AA - cdist_1000AA["zmean"][0:8579])/abs(cdist_1000AA["zmean"][0:8579])

In [135]:
percentile_3AA = np.percentile(rel_error_3AA, [25, 50, 75])*100
percentile_5AA = np.percentile(rel_error_5AA, [25, 50, 75])*100
percentile_10AA = np.percentile(rel_error_10AA, [25, 50, 75])*100
percentile_50AA = np.percentile(rel_error_50AA, [25, 50, 75])*100
percentile_100AA = np.percentile(rel_error_100AA, [25, 50, 75])*100
percentile_500AA = np.percentile(rel_error_500AA, [25, 50, 75])*100
percentile_1000AA = np.percentile(rel_error_1000AA, [25, 50, 75])*100

In [132]:
print("                Carbonaceous ")
print("===============================================")
print("Error")
print("size \t 25% \t 50% \t 75%  ")
print("%.1f &\t %.1f &\t %f &\t %f \\"%(3.5, percentile_3AA[0], percentile_3AA[1], percentile_3AA[2]))
print("%.1i &\t %.1f &\t %f &\t %f \\"%(5,percentile_5AA[0], percentile_5AA[1], percentile_5AA[2]))
print("%.1i &\t %.1f &\t %f &\t %f \\"%(10,percentile_10AA[0], percentile_10AA[1], percentile_10AA[2]))
print("%.1i &\t %.1f &\t %f &\t %f \\"%(50,percentile_50AA[0], percentile_50AA[1], percentile_50AA[2]))
print("%.1i &\t %.1f &\t %f &\t %f \\"%(100,percentile_100AA[0], percentile_100AA[1], percentile_100AA[2]))
print("%.1i &\t %.1f &\t %f &\t %f \\"%(500,percentile_500AA[0], percentile_500AA[1], percentile_500AA[2]))
print("%.1i &\t %.1f &\t %f &\t %f \\"%(1000, percentile_1000AA[0], percentile_1000AA[1], percentile_1000AA[2]))

                Carbonaceous 
Error
size 	 25% 	 50% 	 75%  
3.5 &	 2.0 &	 5.326645 &	 10.175194 \
5 &	 2.1 &	 5.329620 &	 9.487776 \
10 &	 1.6 &	 3.969410 &	 8.073490 \
50 &	 5.9 &	 14.827522 &	 30.006682 \
100 &	 1.1 &	 3.095572 &	 13.509870 \
500 &	 1.1 &	 2.770164 &	 14.171434 \
1000 &	 2.7 &	 7.191845 &	 16.321440 \
